<center>
<h2> HR Analytics </h3>
</center>

### Install packages

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import scipy.stats as stats
import pandas_profiling   #need to install using anaconda prompt (pip install pandas_profiling)

%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 7.5
plt.rcParams['axes.grid'] = True
plt.gray()

from matplotlib.backends.backend_pdf import PdfPages

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor

C:\Users\Kritika\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<Figure size 720x540 with 0 Axes>

In [3]:
# Load the data
HR_Data = pd.read_csv(r"D:\Analytix\Python Learning\Class notes\Class-9-10 Files\3. Logistic Regression\2. HR Analytics - Predicting employee Attrition\HR_comma_sep.csv")

In [4]:
HR_Data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [5]:
profile_report = pandas_profiling.ProfileReport(HR_Data)
profile_report.to_file('HRprofileReport.html')

In [6]:
HR_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   department             14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [7]:
HR_Data.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'department', 'salary'],
      dtype='object')

In [8]:
# subset whole data in cat and num 
Num_Col = HR_Data.dtypes[HR_Data.dtypes != "object"].index.to_list()
Cat_Col = HR_Data.dtypes[HR_Data.dtypes == "object"].index.to_list()
HR_Data_Num = HR_Data[Num_Col]
HR_Data_Cat = HR_Data[Cat_Col]


In [9]:
HR_Data_Cat1 = HR_Data[Cat_Col]

In [10]:
HR_Data_Num.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
0,0.38,0.53,2,157,3,0,1,0
1,0.80,0.86,5,262,6,0,1,0
2,0.11,0.88,7,272,4,0,1,0
3,0.72,0.87,5,223,5,0,1,0
4,0.37,0.52,2,159,3,0,1,0


In [11]:
HR_Data_Cat.head()

,department,salary
0,sales,low
1,sales,medium
2,sales,medium
3,sales,low
4,sales,low


In [12]:
#handling missings & outliers


#handling outliers in num vars

def outliers_num(x):
    x = x.clip(lower = x.quantile(0.01),upper = x.quantile(0.99))
    return x

HR_Data_Num = HR_Data_Num.apply(lambda x : outliers_num(x))

In [13]:
# Converting Catogarical var in num
HR_Data_Cat = pd.get_dummies(HR_Data_Cat,Cat_Col,drop_first=True)


In [14]:
#ST_cat = stores.StoreType.astype("category")
HR_Data_Cat1.department = HR_Data_Cat1.department.astype("category")
HR_Data_Cat1.salary = HR_Data_Cat1.salary.astype("category")

C:\Users\Kritika\Anaconda3\lib\site-packages\pandas\core\generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [15]:
#HR_Data_Cat1.department.cat.codes.nunique

In [16]:
HR_Data_Final = pd.concat([HR_Data_Num,HR_Data_Cat],axis=1)

In [17]:
HR_Data_Final1 = pd.concat([HR_Data_Num,HR_Data_Cat1],axis=1)

In [18]:
HR_Data_Final.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department_RandD,department_accounting,department_hr,department_management,department_marketing,department_product_mng,department_sales,department_support,department_technical,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,1,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,1,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,1,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,1,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,1,0,0,1,0


In [19]:
HR_Data_Final1.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [20]:
# Variable Reduction

In [21]:
#Information value calculation
def calculate_woe_iv(dataset, feature, target):
    lst = []
    for i in range(dataset[feature].nunique()):
        val = list(dataset[feature].unique())[i]
        lst.append({
            'Value': val,
            'All': dataset[dataset[feature] == val].count()[feature],
            'Good': dataset[(dataset[feature] == val) & (dataset[target] == 0)].count()[feature],
            'Bad': dataset[(dataset[feature] == val) & (dataset[target] == 1)].count()[feature]
        })
        
    dset = pd.DataFrame(lst)
    dset['Distr_Good'] = dset['Good'] / dset['Good'].sum()
    dset['Distr_Bad'] = dset['Bad'] / dset['Bad'].sum()
    dset['WoE'] = np.log(dset['Distr_Good'] / dset['Distr_Bad'])
    dset = dset.replace({'WoE': {np.inf: 0, -np.inf: 0}})
    dset['IV'] = (dset['Distr_Good'] - dset['Distr_Bad']) * dset['WoE']
    iv = dset['IV'].sum()
    
    dset = dset.sort_values(by='WoE')
    
    return dset, iv

In [22]:
for col in HR_Data_Final.columns:
    if col == 'left': continue
    else:
        print('WoE and IV for column: {}'.format(col))
        df, iv = calculate_woe_iv(HR_Data_Final, col, 'left')
        print(df)
        print('IV score: {:.2f}'.format(iv))
        print('\n')

WoE and IV for column: satisfaction_level


C:\Users\Kritika\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


    Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE        IV
0    0.38  189    35  154    0.003063   0.043125 -2.644825  0.105958
17   0.40  209    40  169    0.003500   0.047326 -2.604240  0.114132
4    0.37  241    47  194    0.004113   0.054327 -2.580931  0.129598
24   0.39  175    35  140    0.003063   0.039205 -2.549515  0.092145
21   0.43  224    46  178    0.004025   0.049846 -2.516363  0.115302
..    ...  ...   ...  ...         ...        ...       ...       ...
72   0.62  188   186    2    0.016276   0.000560  3.369379  0.052952
68   0.49  209   207    2    0.018113   0.000560  3.476351  0.061022
77   0.68  162   161    1    0.014088   0.000280  3.918184  0.054103
78   0.67  177   176    1    0.015401   0.000280  4.007263  0.060593
76   0.65  199   198    1    0.017326   0.000280  4.125046  0.070315

[91 rows x 8 columns]
IV score: 2.05


WoE and IV for column: last_evaluation
    Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE        IV
22   0.46  211    49  16

C:\Users\Kritika\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\Users\Kritika\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


     Value  All  Good  Bad  Distr_Good  Distr_Bad       WoE        IV
34     127   72    15   57    0.001313   0.015962 -2.498222  0.036597
54     129   63    16   47    0.001400   0.013162 -2.240780  0.026355
24     128   65    18   47    0.001575   0.013162 -2.122997  0.024598
53     130   59    17   42    0.001488   0.011761 -2.067677  0.021243
18     282   36    11   25    0.000963   0.007001 -1.984201  0.011981
..     ...  ...   ...  ...         ...        ...       ...       ...
146    188   73    72    1    0.006300   0.000280  3.113445  0.018744
152    165   78    77    1    0.006738   0.000280  3.180585  0.020540
139    196   79    78    1    0.006825   0.000280  3.193488  0.020902
147    178   85    84    1    0.007350   0.000280  3.267596  0.023103
145    169   86    85    1    0.007438   0.000280  3.279430  0.023474

[198 rows x 8 columns]
IV score: 0.89


WoE and IV for column: time_spend_company
   Value   All  Good   Bad  Distr_Good  Distr_Bad       WoE        IV
3      

In [23]:
#imp var from IV in order are satisfaction_level,number_project,last_evaluation,time_spend_company,average_montly_hours,


In [24]:
#by univariate analysis

In [25]:
gini_df = pd.DataFrame()
for col in HR_Data_Final.columns.difference(['left']):
    model = smf.logit('left~'+str(col), data=HR_Data_Final).fit()
    gini = 2*metrics.roc_auc_score(HR_Data_Final.left, model.predict(HR_Data_Final))-1
    temp = pd.DataFrame([col, gini]).T
    gini_df = pd.concat([gini_df, temp], axis=0)

Optimization terminated successfully.
         Current function value: 0.534317
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.546452
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.547669
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.548747
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.548478
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.547682
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.548859
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.548798
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.548811
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.548803
  

In [26]:
gini_df.columns= ['Feature', 'SomerceD']
gini_df.sort_values(by='SomerceD', ascending=False, inplace=True)

In [29]:
gini_df.to_csv('gini_df3.csv')

In [30]:
# Imp var from Univariate analysis: satisfaction_level,time_spend_company,salary_low,Work_accident,salary_medium

In [31]:
#RFE
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
X = HR_Data_Final[HR_Data_Final.columns.difference(['left'])]
y = HR_Data_Final1[['left']]

rfe = RFE(RandomForestClassifier(), 3)
rfe = rfe.fit(X, y)

X.columns[rfe.get_support()]

C:\Users\Kritika\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Kritika\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Kritika\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Kritika\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Kritika\Anaconda3\lib\sit

Index(['number_project', 'satisfaction_level', 'time_spend_company'], dtype='object')

In [32]:
#SelectKbest
from sklearn.feature_selection import SelectKBest, f_classif
SKB = SelectKBest(f_classif, k=3).fit(X, y )

X.columns[SKB.get_support()]

C:\Users\Kritika\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Index(['Work_accident', 'satisfaction_level', 'time_spend_company'], dtype='object')

In [60]:
Final_list = [
    'satisfaction_level',
    #'number_project',
    #'last_evaluation',
    'time_spend_company',
    #'average_montly_hours',
    #'salary_low',
    'Work_accident',
    #'salary_medium'
            ]

In [61]:
X_new = X[Final_list]

In [62]:
pd.DataFrame(X_new.values)

,0,1,2
0,0.38,3.0,0.0
1,0.80,6.0,0.0
2,0.11,4.0,0.0
3,0.72,5.0,0.0
4,0.37,3.0,0.0
...,...,...,...
14994,0.40,3.0,0.0
14995,0.37,3.0,0.0
14996,0.37,3.0,0.0
14997,0.11,4.0,0.0


In [63]:
### VIF Calculation for variables
vif = pd.DataFrame()
vif["VIF_Factor"] = [variance_inflation_factor(X_new.values, i) for i in range(X_new.shape[1])]
vif["features"] = X_new.columns

In [64]:
vif.sort_values(by='VIF_Factor',ascending=False)

,VIF_Factor,features
0,3.489043,satisfaction_level
1,3.428153,time_spend_company
2,1.168202,Work_accident


In [65]:
data_final = pd.concat([X_new, y], axis=1)

In [66]:
#split the data into train & test
train, test = train_test_split(data_final, test_size=0.3, random_state=123)

In [67]:
'left~'+'+'.join(train.columns.difference(['left']))

'left~Work_accident+satisfaction_level+time_spend_company'

In [68]:
formula = 'left~Work_accident+satisfaction_level+time_spend_company'

In [69]:
model = smf.logit(formula, data=train).fit()

Optimization terminated successfully.
         Current function value: 0.453441
         Iterations 7


In [70]:
print(model.summary())

                           Logit Regression Results                           
Dep. Variable:                   left   No. Observations:                10499
Model:                          Logit   Df Residuals:                    10495
Method:                           MLE   Df Model:                            3
Date:                Fri, 09 Oct 2020   Pseudo R-squ.:                  0.1739
Time:                        02:47:15   Log-Likelihood:                -4760.7
converged:                       True   LL-Null:                       -5762.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.3966      0.087      4.544      0.000       0.226       0.568
Work_accident         -1.5444      0.106    -14.554      0.000      -1.752      -1.336
satisfaction_level  

In [71]:
#Metrics based on the probability - AUC, Gini
train['prob'] = model.predict(train)
test['prob'] = model.predict(test)

C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [72]:
#Calculating AUC for train & Test

In [73]:
train_auc = metrics.roc_auc_score(train.left, train.prob)
train_auc

0.790718139767471

In [74]:
test_auc = metrics.roc_auc_score(test.left, test.prob)
test_auc

0.7877721711801275

In [75]:
#Calculating Gini (somerceD) for train & Test

In [76]:
train_gini = 2*metrics.roc_auc_score(train.left, train.prob)-1
train_gini

0.5814362795349419

In [77]:
test_gini = 2*metrics.roc_auc_score(test.left, test.prob)-1
test_gini

0.5755443423602551

In [78]:
temp = train
roc_df = pd.DataFrame()
for cut_off in np.linspace(0,1):
    temp['y_pred'] = np.where(train.prob>cut_off, 1, 0)
    temp['TP'] = np.where(((train.left ==1) & (train.y_pred==1)), 1,0)
    temp['TN'] = np.where(((train.left ==0) & (train.y_pred==0)), 1,0)
    temp['FP'] = np.where(((train.left ==0) & (train.y_pred==1)), 1,0)
    temp['FN'] = np.where(((train.left ==1) & (train.y_pred==0)), 1,0)
    sensitivity = temp.TP.sum()/(temp.TP.sum()+temp.FN.sum())
    specificity = temp.TN.sum()/(temp.TN.sum()+temp.FP.sum())
    accuracy = (temp.TN.sum()+temp.TP.sum())/(temp.y_pred.count())
    temp_df = pd.DataFrame([cut_off, sensitivity, specificity, accuracy]).T
    temp_df.columns = ['cutoff', 'sensitivity', 'specificity', 'accuracy']
    roc_df = pd.concat([roc_df, temp_df], axis=0)

C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

In [79]:
roc_df['total'] = roc_df.sensitivity + roc_df.specificity

In [80]:
roc_df.sort_values(by = 'total', ascending=False).head(1)

,cutoff,sensitivity,specificity,accuracy,total
0,0.306122,0.6924,0.84323,0.807315,1.53563


In [81]:
roc_df[roc_df.total == roc_df.total.max()]

,cutoff,sensitivity,specificity,accuracy,total
0,0.306122,0.6924,0.84323,0.807315,1.53563


In [82]:
# so cut off is 0.306122

In [83]:
train['default_pred'] = np.where(train.prob> 0.306122, 1, 0)
test['default_pred'] = np.where(test.prob> 0.306122, 1, 0)

C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Kritika\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [84]:
#Good ness of fit metrics based on categorical predicted output for train & test

print(metrics.classification_report(train.left, train.default_pred))

              precision    recall  f1-score   support

           0       0.90      0.84      0.87      7999
           1       0.58      0.69      0.63      2500

   micro avg       0.81      0.81      0.81     10499
   macro avg       0.74      0.77      0.75     10499
weighted avg       0.82      0.81      0.81     10499



In [85]:
print(metrics.classification_report(test.left, test.default_pred))

              precision    recall  f1-score   support

           0       0.90      0.84      0.87      3429
           1       0.57      0.70      0.63      1071

   micro avg       0.80      0.80      0.80      4500
   macro avg       0.74      0.77      0.75      4500
weighted avg       0.82      0.80      0.81      4500



In [ ]:
train.head()

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'n_neighbors' : [3,5,7,9,11,13], 'weights': ['uniform', 'distance']}

In [ ]:
HR_Data_Final.head()

In [ ]:
X = HR_Data_Final[HR_Data_Final.columns.difference(['left'])]
y = HR_Data_Final.left

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size=0.3, random_state= 123)

In [ ]:
#pre-requisite for KNN - standardization
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(train_X)

In [ ]:
train_std = pd.DataFrame(sc.transform(train_X), columns = train_X.columns)

In [ ]:
test_std = pd.DataFrame(sc.transform(test_X), columns = test_X.columns)

In [ ]:
#Tuning parameters
#n_neighbors= [3,4,5,6,7,8,9,10]
#weights=['uniform', 'distance']

param_grid = {'n_neighbors': [3,4,5,6,7,8,9,10], 'weights':['uniform', 'distance']}

model = GridSearchCV(KNeighborsClassifier(),param_grid, cv =5, scoring = 'roc_auc')

In [ ]:
model.fit(train_std, train_y)

In [ ]:
model.best_params_

In [ ]:
model.best_score_

In [ ]:
#K- Nearest Nieghbors

model = KNeighborsClassifier(n_neighbors=10, weights='distance')
model.fit(train_std, train_y)
print(metrics.classification_report(train_y, model.predict(train_std)))
print(metrics.classification_report(test_y, model.predict(test_std)))

In [ ]:
print(metrics.roc_auc_score(train_y, pd.DataFrame(model.predict_proba(train_std))[1]))

print(metrics.roc_auc_score(test_y, pd.DataFrame(model.predict_proba(test_std))[1]))

In [ ]:
#Decision Trees
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz

In [ ]:
#splitting criterion = 'gini', max_depth = 5, max_leaf_nodes=10
model = DecisionTreeClassifier(max_depth=5, max_leaf_nodes=10)
model.fit(train_X, train_y)

In [ ]:
print(metrics.roc_auc_score(train_y, pd.DataFrame(model.predict_proba(train_X))[1]))

print(metrics.roc_auc_score(test_y, pd.DataFrame(model.predict_proba(test_X))[1]))

In [ ]:
print('Training Classification report:')
print(metrics.classification_report(train_y, model.predict(train_X)))
print('Testing Classification report:')
print(metrics.classification_report(test_y, model.predict(test_X)))

In [ ]:
#Scenario-2: If we don't know best parameters
#Tuning parameters
#max_depth= [5,6,7,8,9,10]
#max_leaf_nodes=[5,6,7,8,9,10]

param_grid = {'max_depth': [5,6,7,8,9,10], 'max_leaf_nodes':[5,6,7,8,9,10]}

model = GridSearchCV(DecisionTreeClassifier(),param_grid, cv =5, scoring = 'roc_auc')

In [ ]:
model.fit(train_X, train_y)

In [ ]:
model.best_params_

In [ ]:
model = DecisionTreeClassifier(max_depth=5, max_leaf_nodes=10)
model.fit(train_X, train_y)

In [ ]:
print(metrics.roc_auc_score(train_y, pd.DataFrame(model.predict_proba(train_X))[1]))

print(metrics.roc_auc_score(test_y, pd.DataFrame(model.predict_proba(test_X))[1]))

In [ ]:
print('Training Classification report:')
print(metrics.classification_report(train_y, model.predict(train_X)))
print('Testing Classification report:')
print(metrics.classification_report(test_y, model.predict(test_X)))

In [ ]:
import os     
os.environ["PATH"] += os.pathsep + 'D:/Analytix/Python Learning/Class notes/Class_13-14_Files/graphviz-2.38/release/bin'


In [ ]:
import pydotplus as pdot
from sklearn.tree import export_graphviz

In [ ]:
# Exporting the tree output in the form opendocument
export_graphviz( model,
              out_file = "model_tree.odt",
              feature_names = train_X.columns )

In [ ]:
chd_tree_graph = pdot.graphviz.graph_from_dot_file( 'model_tree.odt' )

In [ ]:
chd_tree_graph.write_jpg( 'model_tree1.jpg' )